In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llamamodel3epoch/llamamodel/adapter_model.safetensors
/kaggle/input/llamamodel3epoch/llamamodel/adapter_config.json
/kaggle/input/llm-weights-3epoch/adapter_model.safetensors
/kaggle/input/api-dataset/api_dataset/train.json
/kaggle/input/api-dataset/api_dataset/test.json
/kaggle/input/api-dataset/api_dataset/val.json


In [2]:
# 1. Uninstall existing conflicting libraries
!pip uninstall -y transformers peft trl bitsandbytes accelerate triton

# 2. Install the "Golden Combination" for Kaggle T4
!pip install -q -U \
  torch==2.3.0 \
  transformers==4.40.0 \
  peft==0.10.0 \
  trl==0.8.6 \
  accelerate==0.30.0 \
  bitsandbytes==0.42.0 \
  datasets==2.19.0
  
# Note: We skip installing 'triton' explicitly because bitsandbytes 0.42 
# will use the built-in CUDA kernels instead of Triton if it's messy.
# This prevents the 'triton.ops' error entirely.


Found existing installation: transformers 4.57.1
Uninstalling transformers-4.57.1:
  Successfully uninstalled transformers-4.57.1
Found existing installation: peft 0.17.1
Uninstalling peft-0.17.1:
  Successfully uninstalled peft-0.17.1
Found existing installation: accelerate 1.11.0
Uninstalling accelerate-1.11.0:
  Successfully uninstalled accelerate-1.11.0
Found existing installation: triton 3.4.0
Uninstalling triton-3.4.0:
  Successfully uninstalled triton-3.4.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 3.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 240.9 kB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 2.8 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━

In [3]:
pip install evaluate rouge_score nltk

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.2 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=caa627fa0275d9b24859a8eeb40d1189c842125fe74e899a66106b5e72e2aa0d
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score
Note: you may need to restart the kernel to use updated packages.


In [4]:
import torch
import pandas as pd
import numpy as np
from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    BitsAndBytesConfig, 
    TrainingArguments,
    pipeline
)
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer
import evaluate
import nltk

# --- CONFIGURATION ---
MODEL_ID = "unsloth/llama-3-8b-Instruct-bnb-4bit" # Pre-quantized base model
OUTPUT_DIR = "llama3-openapi-adapter"
MAX_SEQ_LENGTH = 512  # Increased to 512 to be safe for longer docs


2025-12-22 16:29:53.263957: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766420993.453653      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766420993.508200      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766420993.959690      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766420993.959739      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766420993.959742      55 computation_placer.cc:177] computation placer alr

In [5]:
# Load local JSON files into HuggingFace Dataset format
dataset = load_dataset("json", data_files={
    "train": "/kaggle/input/api-dataset/api_dataset/train.json",
    "validation": "/kaggle/input/api-dataset/api_dataset/val.json",
    "test": "/kaggle/input/api-dataset/api_dataset/test.json"
})

print("✅ Data Loaded:")
print(f"   Train: {len(dataset['train'])} examples")
print(f"   Val:   {len(dataset['validation'])} examples")
print(f"   Test:  {len(dataset['test'])} examples")

# Example check
print("\n🔍 Sample Input:")
print(dataset['train'][0]['input_text'])
print("🔍 Sample Target:")
print(dataset['train'][0]['target_text'])


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

✅ Data Loaded:
   Train: 6341 examples
   Val:   793 examples
   Test:  793 examples

🔍 Sample Input:
Method: GET | Path: /V1/companyCredits/company/{companyId} | Summary: companyCredits/company/{companyId} | Tags: companyCredits/company/{companyId}
🔍 Sample Target:
Returns data on the credit limit for a specified company.


In [ ]:
# 4-bit Quantization Config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token # Fix for Llama-3 padding
tokenizer.padding_side = "right" # Fix for FP16 training

# Load Model
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    use_cache=False # Disable cache during training
)
model.config.pretraining_tp = 1 

print("✅ Model Loaded in 4-bit!")


In [ ]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache()


In [ ]:
# --- 1. LoRA CONFIGURATION ---
peft_config = LoraConfig(
    r=8,        
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)

# --- 2. FORMATTING FUNCTION ---
# Converts your input/target pairs into the Llama-3 chat format
def formatting_prompts_func(examples):
    texts = []
    for input_text, target_text in zip(examples['input_text'], examples['target_text']):
        # Llama-3 Prompt Template
        prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an expert technical writer. Write a concise, professional API description.<|eot_id|><|start_header_id|>user<|end_header_id|>

{input_text}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{target_text}<|eot_id|>"""
        texts.append(prompt)
    return texts

# --- 3. MEMORY-OPTIMIZED TRAINING ARGUMENTS ---
args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=3,
    per_device_train_batch_size=2,  # REDUCED: 4 -> 2 (Critical for T4)
    gradient_accumulation_steps=16,  # INCREASED: 4 -> 8 (Keeps effective batch size = 16)
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    logging_steps=25,
    save_strategy="epoch",
    evaluation_strategy="steps",
    eval_steps=100,
    report_to="none",
    
    # --- CRITICAL MEMORY FIXES ---
    gradient_checkpointing=True,    # SAVES ~50% VRAM (Trades speed for memory)
    optim="paged_adamw_8bit",       # SAVES VRAM (Pager optimizer moves states to CPU if needed)
    max_grad_norm=0.3,              # Prevents gradient explosions consuming memory
    warmup_ratio=0.03,
    group_by_length=True,           # Groups similar length prompts (saves padding memory)
)

# --- 4. TRAINER WITH GRADIENT CHECKPOINTING ---
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    peft_config=peft_config,
    formatting_func=formatting_prompts_func,
    max_seq_length=256,
    tokenizer=tokenizer,
    args=args,
    packing=False
)

# Enable gradient checkpointing explicitly on the model
model.config.use_cache = False  # Silence warnings during training
trainer.model.print_trainable_parameters() 

print("🚀 Starting MEMORY-SAFE QLoRA Fine-Tuning...")
trainer.train()

# Save final adapter
trainer.model.save_pretrained("final_lora_adapter")
print("✅ Adapter Saved to 'final_lora_adapter'")


In [ ]:
from tqdm import tqdm

# Load ROUGE
rouge = evaluate.load("rouge")

def generate_response(input_text):
    # Format input exactly like training
    prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an expert technical writer. Write a concise, professional API description.<|eot_id|><|start_header_id|>user<|end_header_id|>

{input_text}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    
    # Generate
    with torch.no_grad():
        outputs = model.generate(
            **inputs, 
            max_new_tokens=128, 
            pad_token_id=tokenizer.eos_token_id,
            do_sample=False # Greedy decoding for reproducibility
        )
    
    # Extract only the response (remove the prompt)
    response = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
    return response.strip()

# --- RUN EVALUATION ON TEST SET (Sample 100 for speed, or all for accuracy) ---
print("🚀 Generating predictions on Test Set...")
predictions = []
references = []

# Using first 100 examples for quick feedback (run full set for final report)
test_subset = dataset["test"].select(range(100)) 

for row in tqdm(test_subset):
    pred = generate_response(row["input_text"])
    predictions.append(pred)
    references.append(row["target_text"])

# Compute ROUGE
print("\n📊 Calculating Metrics...")
results = rouge.compute(predictions=predictions, references=references)
print("🏆 QLoRA Results:", results)

# Show examples
print("\n🔍 Example Output:")
print(f"Target: {references[0]}")
print(f"Predicted: {predictions[0]}")


evaluation in new way

In [7]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import evaluate
from tqdm import tqdm

# --- 1. LOAD MODEL (If not already loaded) ---
# If your model is already in memory from training, skip this block.
# If you restarted, run this:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

base_model = AutoModelForCausalLM.from_pretrained(
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    quantization_config=bnb_config,
    device_map="auto"
)

# Load your found weights
# pointing to the folder where adapter_model.bin or safetensors is
model = PeftModel.from_pretrained(base_model, "/kaggle/input/llamamodel3epoch/llamamodel") 
model.eval()

tokenizer = AutoTokenizer.from_pretrained("unsloth/llama-3-8b-Instruct-bnb-4bit")
tokenizer.pad_token = tokenizer.eos_token

# --- 2. OPTIMIZED GENERATION FUNCTION ---
rouge = evaluate.load("rouge")

def generate_response(input_text):
    # Prompt: We explicitly ask for brevity here too
    prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Write a single-sentence description for this API.<|eot_id|><|start_header_id|>user<|end_header_id|>

{input_text}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs, 
            # --- CRITICAL PARAMETERS FOR BREVITY ---
            max_new_tokens=64,        # Force stop after ~40-50 words
            min_length=10,            # Ensure it generates *something*
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            do_sample=False,          # Greedy decoding (most focused)
            repetition_penalty=1.2,   # Punish it for repeating "Reviews are..."
            temperature=0.1,          # Low randomness = more concise
            # Stop tokens (optional): Force stop at newline if needed
            # stopping_criteria=... 
        )
    
    # Decode
    response = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
    
    # Post-processing: Cut off if it starts a new bullet point or paragraph
    if "\n" in response:
        response = response.split("\n")[0]
        
    return response.strip()

# --- 3. RUN EVALUATION ---
print("🚀 Generating Optimized Predictions...")
predictions = []
references = []

# Test on 200 samples (enough for a good estimate)
test_subset = dataset["test"].select(range(200)) 

for row in tqdm(test_subset):
    pred = generate_response(row["input_text"])
    predictions.append(pred)
    references.append(row["target_text"])

# Compute ROUGE
print("\n📊 Calculating Metrics...")
results = rouge.compute(predictions=predictions, references=references)
print("🏆 QLoRA Results (Optimized):", results)

# Check the first few
print("\n🔍 Example Output:")
print(f"Target: {references[0]}")
print(f"Predicted: {predictions[0]}")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.12/dist-packages/transformers/quantizers/auto.py:159: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force 

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


🚀 Generating Optimized Predictions...


  0%|          | 0/200 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
100%|██████████| 200/200 [30:43<00:00,  9.22s/it]



📊 Calculating Metrics...
🏆 QLoRA Results (Optimized): {'rouge1': np.float64(0.3444165847872224), 'rouge2': np.float64(0.2227221811979705), 'rougeL': np.float64(0.29737349321424084), 'rougeLsum': np.float64(0.2967377970714394)}

🔍 Example Output:
Target: Create a review for the individual watchlist screening. Reviews are compliance reports created by users in your organization regarding the relevance of potential hits found by Plaid.
Predicted: Create a review of the results returned by Plaid, and optionally add notes about your decision to allow or reject the customer. Reviews are optional but highly recommended as they provide audit trail information in case you need it later. If reviews are not created, all screenings will be automatically approved after 30 days. Note that if


In [10]:
import evaluate
import numpy as np
from tqdm import tqdm
import torch

# --- 1. SETUP METRICS ---
rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")
bertscore = evaluate.load("bertscore")

print("✅ Metrics Loaded (ROUGE, BLEU, BERTScore)")

# --- 2. GENERATE PREDICTIONS (Using your 200 sample subset) ---
# Assuming 'predictions' and 'references' lists are already populated 
# from your previous run (the one that gave you 34.4%). 
# If not, re-run the generation loop first!

# --- 3. CALCULATE ALL METRICS ---
print("📊 Calculating ALL Metrics...")

# A. ROUGE
rouge_results = rouge.compute(predictions=predictions, references=references)

# B. BLEU
# BLEU expects references as list of lists: [['ref']]
bleu_refs = [[r] for r in references]
bleu_results = bleu.compute(predictions=predictions, references=bleu_refs)

# C. BERTScore (The semantic tie-breaker!)
# lang="en" downloads a roberta-large model (~1.4GB) - might take a minute
bert_results = bertscore.compute(predictions=predictions, references=references, lang="en")
bert_f1 = np.mean(bert_results["f1"]) # Average F1 across all samples

# --- 4. PRINT FINAL TABLE ---
print("\n" + "="*40)
print("🏆 FINAL QLoRA EVALUATION RESULTS")
print("="*40)
print(f"ROUGE-1:      {rouge_results['rouge1']*100:.2f}")
print(f"ROUGE-2:      {rouge_results['rouge2']*100:.2f}")
print(f"ROUGE-L:      {rouge_results['rougeL']*100:.2f}")
print("-" * 40)
print(f"BLEU:         {bleu_results['bleu']*100:.2f}")
print("-" * 40)
print(f"BERTScore F1: {bert_f1*100:.2f}")  # This should be high (85%+)
print("="*40)


✅ Metrics Loaded (ROUGE, BLEU, BERTScore)
📊 Calculating ALL Metrics...


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



🏆 FINAL QLoRA EVALUATION RESULTS
ROUGE-1:      34.44
ROUGE-2:      22.27
ROUGE-L:      29.74
----------------------------------------
BLEU:         22.34
----------------------------------------
BERTScore F1: 87.54
